In [9]:
## old, remove
import torch.nn as nn

class EClassifier(nn.Module):
    def __init__(self, num_locations, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(num_locations, embedding_dim)
        self.linear = nn.Linear(embedding_dim, num_classes)
    def forward(self, x):
        # x is a tensor containing word ids
        loc_emb = self.embedding(x[:,0])
        return self.linear(torch.cat((loc_emb, x[:,1:]), 1))

In [58]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder

data_dir = "../data/"
train_vals_df = pd.read_csv(os.path.join(data_dir, "train_values.csv"))
train_labels_df = pd.read_csv(os.path.join(data_dir, "train_labels.csv"))

# get categorical data
X1 = train_vals_df["geo_level_1_id"].to_numpy()

## get numerical data
df = train_vals_df[[
'count_floors_pre_eq',
'age',
'area_percentage',
'height_percentage']]

# normalize numerical data
min_max_scaler = preprocessing.MinMaxScaler()
X2 = min_max_scaler.fit_transform(df.values)


# get labels

y = train_labels_df["damage_grade"].to_numpy()
y = np.eye(3)[y-1]

X = np.concatenate((X1[:,np.newaxis],X2), axis = 1)

## Train val test split
### X1 is categorical data and X2 is numerical data

In [59]:
rng = np.random.RandomState(1)
perm = rng.permutation(range(len(X)))
X_rand = X[perm]
y_rand = y[perm]

trainIdx = int(.8*len(X))
X_train = X[:trainIdx]
y_train = y[:trainIdx]
X_val = X[trainIdx:]
y_val = y[trainIdx:]


## Create model that embeds categorical data

In [65]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense, Embedding

## Creating the layers
batch_size = 100
input = Input(shape=(5,)) 
embedding = Embedding(31, 2)(input[:,0])
Layer_1 = Dense(10, activation="relu")(tf.concat([embedding, input[:,1:]], 1))
Layer_2 = Dense(5, activation="relu")(Layer_1)
output_layer= Dense(3, activation="softmax")(Layer_2)

model = tf.keras.Model(inputs=input, outputs=output_layer, name="emb_model")
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='CategoricalCrossentropy', metrics=['Accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val))

Epoch 1/50
2085/2085 [==============================] - 1s 541us/step - loss: 0.8722 - accuracy: 0.0000e+00 - val_loss: 0.7750 - val_accuracy: 0.0000e+00
Epoch 2/50
2085/2085 [==============================] - 1s 491us/step - loss: 0.7678 - accuracy: 0.0000e+00 - val_loss: 0.7628 - val_accuracy: 0.0000e+00
Epoch 3/50
2085/2085 [==============================] - 1s 497us/step - loss: 0.7576 - accuracy: 0.0000e+00 - val_loss: 0.7554 - val_accuracy: 0.0000e+00
Epoch 4/50
2085/2085 [==============================] - 1s 524us/step - loss: 0.7518 - accuracy: 0.0000e+00 - val_loss: 0.7510 - val_accuracy: 0.0000e+00
Epoch 5/50
2085/2085 [==============================] - 1s 490us/step - loss: 0.7471 - accuracy: 0.0000e+00 - val_loss: 0.7481 - val_accuracy: 0.0000e+00
Epoch 6/50
2085/2085 [==============================] - 1s 494us/step - loss: 0.7466 - accuracy: 0.0000e+00 - val_loss: 0.7462 - val_accuracy: 0.0000e+00
Epoch 7/50
2085/2085 [==============================] - 1s 490us/step - loss